In [4]:
import bs4
import fitz # pdf reader
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings



In [13]:
import fitz  # PyMuPDF
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document
import openai

def read_pdf(file_path):
    # Open the PDF file
    document = fitz.open(file_path)
    pdf_text = ""

    # Iterate through the pages and extract text
    for page_num in range(len(document)):
        page = document.load_page(page_num)
        pdf_text += page.get_text()

    return pdf_text

# Example usage
pdf_file_path = "example.pdf"
docs = read_pdf(pdf_file_path)

# Convert the text to a Document object
document = Document(page_content=docs, metadata={"source": pdf_file_path})

# Split the document into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents([document])

# Create the vector store
try:
    embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002")
    vectorstore = Chroma.from_documents(documents=splits, embedding=embedding_model)
except openai.error.PermissionDeniedError:
    print("Access to the specified model is denied. Trying a different model...")
    # Use a different model
    embedding_model = OpenAIEmbeddings(model="text-embedding-babbage-001")
    vectorstore = Chroma.from_documents(documents=splits, embedding=embedding_model)

# Create the retriever
retriever = vectorstore.as_retriever()

# Example query
query = "What are the benefits of effective recycling programs?"
results = retriever.get_relevant_documents(query)
for result in results:
    print(result.page_content)


You’re not covered for
8 Money.
8 Credit or debit cards.
8 Stamps.
8 Tickets.
8 Vouchers.
8 Documents.
8 Securities (such as share or Premium 
Bond certificates).
8 Goods or samples for a trade or business.
8 Any property that’s insured under any 
other policy.
Vandalism Promise
Included with:
Comp
Comp+
If you claim for damage to your car that’s 
a result of vandalism, this won’t affect the 
No Claim Discount on this policy.
What happens if you claim
If you claim, you’ll need to:
> Pay the excess that applies to accidental 
damage claims when the car is unattended.
> Report the vandalism to the police. To do this, 
call 101 (unless it’s an emergency).
> Get a crime reference number from the 
police and give it to us.
Until we have the crime reference number, you 
may temporarily lose the No Claim Discount on 
this policy. 
You’re not covered for
8 Our Vandalism Promise doesn’t cover 
damage caused by another vehicle.
Section 6: Personal benefits continued
Page 23
You’re not covered fo

In [ ]:
#### RETRIEVAL and GENERATION ####

# Prompt
prompt = hub.pull("rlm/rag-prompt")

# LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Question
rag_chain.invoke("What is Task Decomposition?")